In [5]:
import datetime

import fiona
from shapely.geometry import shape, mapping
from pystac_client import Client
import planetary_computer as pc

from nrt.validate.interface import SegmentsLabellingInterface
from nrt.validate.loaders import STACLoader
from nrt.validate.webmaps import EsriWaybackBasemap, PlanetBasemap, GoogleBasemap
from nrt.validate import utils
from nrt.validate.indices import *
from nrt.validate.composites import *
from nrt.validate.xr_transforms import *


with fiona.open('/home/loic/git/nrt-data/zenodo/germany_2021_30-samples.fgb') as con:
    crs = con.crs
    fc = [{'geometry': mapping(shape(feat['geometry']).buffer(10, cap_style=3)),
           'properties': {'pid': idx}} for idx,feat in enumerate(con, start=1)]

# webmap = PlanetBasemap(frequency='quarterly',
#                        begin=datetime.datetime(2021, 1, 1),
#                        end=datetime.datetime.now(),
#                        api_key='XXX').map
webmap = EsriWaybackBasemap().map
# webmap = GoogleBasemap().map

catalog = Client.open('https://planetarycomputer.microsoft.com/api/stac/v1',
                      modifier=pc.sign_inplace)
collection_id = 'sentinel-2-l2a'
bands = ['B02', 'B03', 'B04', 'B08', 'B11', 'B12', 'SCL']
resampling = {band: 'nearest' if band == 'SCL' else 'cubic' for band in bands}
dt = [datetime.datetime(2019, 1, 1), datetime.datetime(2021,12,31)]
vis = {'NDVI': utils.combine_transforms(S2CloudMasking(), NDVI(nir='B08')),
       'CR-SWIR': utils.combine_transforms(S2CloudMasking(), CR_SWIR(nir='B08'))}
window_size = 600
compositor = S2CIR(r='B08')
query = {"eo:cloud_cover": {"lt": 20}}
res = 10
prefetch = 3


# Loader
loader = STACLoader(fc=fc,
                    key='pid',
                    crs=crs,
                    client=catalog,
                    collection_id=collection_id,
                    bands=bands,
                    resampling=resampling,
                    datetime=dt,
                    vis=vis,
                    window_size=window_size,
                    compositor=compositor,
                    query=query,
                    res=res,
                    prefetch=prefetch)



# Interface
i = SegmentsLabellingInterface(loader=loader,
                               webmap=webmap,
                               res=None,
                               labels=['Non-treed',
                                       'Stable tree cover',
                                       'Dieback',
                                       'Regrowth'])


In [ ]:
i.display()